# Radial Distribution Function (RDF) Calculation in Liquid Argon

This notebook simulates a three-dimensional system of Argon atoms interacting via the Lennard-Jones (LJ) potential and computes the Radial Distribution Function (RDF) $g(r)$ to characterize the liquid structure.

---

## Configuration Parameters (passed via `params` dictionary):
- `dimensions`: Simulation dimensionality (fixed to 3D).
- `n_particles`: Number of Argon atoms (default: 1000).
- `density`: Mass density of Argon ($1374 \, \mathrm{kg/m^3}$).
- `dt`: Time step for integration ($2 \times 10^{-15}\,\mathrm{s}$).
- `steps`: Total number of simulation steps (default: 1000).
- `temperature`: Initial and target temperature ($94.4 \, \mathrm{K}$).
- `sigma`, `epsilon`: Lennard-Jones potential parameters for Argon ($\sigma = 3.4 \times 10^{-10}\,\mathrm{m}$, $\epsilon = 1.656 \times 10^{-21}\,\mathrm{J}$).
- `rcutoff`: Cutoff distance for interactions ($2.5\sigma$).
- `tau_ber`: Relaxation time for Berendsen thermostat.
- `gamma_langevin`: Friction constant for Langevin thermostat.
- `thermostat_type`: Choice of thermostat (`"berendsen"` or `"langevin"`).

---

## Key Functions and Return Values:
- `compute_lj_force(r, sigma, epsilon, rcutoff)`: Returns the Lennard-Jones force for a given distance.
- `compute_lj_potential(r, sigma, epsilon, rcutoff)`: Returns the truncated-shifted Lennard-Jones potential energy.
- `build_linked_cells(positions, box_size, rcutoff)`: Constructs neighbor lists using the linked-cell algorithm for efficient force calculations.
- `compute_forces_lca(...)`: Computes interatomic forces with periodic boundary conditions (PBC).
- `apply_berendsen_thermostat(velocities, target_temp, current_temp, dt, tau)`: Applies Berendsen thermostat to adjust velocities.
- `apply_langevin_thermostat(velocities, gamma, dt, target_temp)`: Applies Langevin thermostat to regulate system temperature with noise and friction.
- `create_lattice(n_particles, box_size, dimensions)`: Initializes particle positions on a slightly perturbed cubic lattice.
- `initialize_velocities(n_particles, dimensions, target_temp)`: Assigns initial velocities according to a Maxwell-Boltzmann distribution.
- `compute_rdf(positions, box_size, rcutoff, n_bins)`: Computes the radial distribution function $g(r)$ from atomic positions.
- `run_simulation(params, compute_rdf_flag, rdf_sample_steps, n_bins)`: Main loop to perform molecular dynamics simulation and optionally accumulate RDF data.

---

## Simulation Workflow:
- Initialize a periodic box containing 1000 Argon atoms.
- Use the Lennard-Jones potential truncated and shifted at $r_\text{cutoff} = 2.5\sigma$.
- Integrate equations of motion with the velocity-Verlet scheme.
- Apply the Langevin or Berendsen thermostat to regulate temperature.
- Collect samples of the RDF during the final steps of the simulation.
- Average the RDF over the sampling period to improve statistical reliability.

---

## Output:
- Plots the **Radial Distribution Function** $g(r)$ vs $r$.
- $r$ is converted to Angstroms for easy comparison with experimental and literature data.
- $g(r)$ characterizes the liquid structure, showing typical nearest-neighbor peaks and long-range decay toward 1.

---

## Parameters:
| Parameter         | Value                      |
|-------------------|-----------------------------|
| dimensions        | 3                           |
| n_particles       | 1000                        |
| density           | 1374 $\mathrm{kg/m^3}$       |
| dt                | $2 \times 10^{-15}\,\mathrm{s}$ |
| steps             | 1000                        |
| temperature       | 94.4 $\mathrm{K}$            |
| sigma             | $3.4 \times 10^{-10}\,\mathrm{m}$ |
| epsilon           | $1.656 \times 10^{-21}\,\mathrm{J}$ |
| rcutoff           | $2.5\sigma$                  |
| tau_ber           | $1 \times 10^{-13}\,\mathrm{s}$ |
| gamma_langevin    | $1 \times 10^{13}\,\mathrm{s^{-1}}$ |

---


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  


sigma = 3.4e-10  # m
epsilon = 1.656e-21  # J
mass_argon = 39.95 * 1.66054e-27  # kg
density = 1374  # kg/m³

def compute_lj_force(r, sigma, epsilon, rcutoff):
    if r >= rcutoff or r < 1e-12:
        return 0.0
    inv_r = sigma / r
    inv_r6 = inv_r ** 6
    inv_r12 = inv_r6 ** 2
    return 24 * epsilon * (2 * inv_r12 - inv_r6) / r

def compute_lj_potential(r, sigma, epsilon, rcutoff):
    if r >= rcutoff:
        return 0.0
    inv_r = sigma / r
    inv_r6 = inv_r ** 6
    inv_r12 = inv_r6 ** 2
    pot = 4 * epsilon * (inv_r12 - inv_r6)
    inv_rcut = sigma / rcutoff
    shift = 4 * epsilon * (inv_rcut ** 12 - inv_rcut ** 6)
    return pot - shift

def build_linked_cells(positions, box_size, rcutoff):
    n_particles, dim = positions.shape
    lc = max(1, int(np.floor(box_size / rcutoff)))
    lc_dim = [lc] * dim
    rc = box_size / lc
    EMPTY = -1
    head = [EMPTY] * (lc ** dim)
    lscl = [EMPTY] * n_particles

    for i in range(n_particles):
        mc = [int(positions[i][a] / rc) for a in range(dim)]
        mc = [min(max(0, idx), lc - 1) for idx in mc]
        if dim == 2:
            c_index = mc[0] * lc_dim[1] + mc[1]
        else:
            c_index = mc[0] * lc_dim[1] * lc_dim[2] + mc[1] * lc_dim[2] + mc[2]
        lscl[i] = head[c_index]
        head[c_index] = i

    return head, lscl, lc_dim

def compute_forces_lca(positions, box_size, rcutoff, sigma, epsilon, use_pbc=True):
    n_particles, dim = positions.shape
    head, lscl, lc_dim = build_linked_cells(positions, box_size, rcutoff)
    EMPTY = -1
    forces = np.zeros_like(positions)
    potential_energy = 0.0
    neighbor_offsets = np.array(np.meshgrid(*[[-1, 0, 1]] * dim)).T.reshape(-1, dim)

    for mc in np.ndindex(*lc_dim):
        if dim == 2:
            c_index = mc[0] * lc_dim[1] + mc[1]
        else:
            c_index = mc[0] * lc_dim[1] * lc_dim[2] + mc[1] * lc_dim[2] + mc[2]
        i = head[c_index]
        while i != EMPTY:
            pos_i = positions[i]
            for offset in neighbor_offsets:
                mc1 = np.array(mc) + offset
                rshift = np.zeros(dim)
                valid_cell = True
                for a in range(dim):
                    if use_pbc:
                        if mc1[a] < 0:
                            mc1[a] += lc_dim[a]
                            rshift[a] = -box_size
                        elif mc1[a] >= lc_dim[a]:
                            mc1[a] -= lc_dim[a]
                            rshift[a] = box_size
                    else:
                        if mc1[a] < 0 or mc1[a] >= lc_dim[a]:
                            valid_cell = False
                            break
                if not valid_cell:
                    continue
                if dim == 2:
                    c1 = mc1[0] * lc_dim[1] + mc1[1]
                else:
                    c1 = mc1[0] * lc_dim[1] * lc_dim[2] + mc1[1] * lc_dim[2] + mc1[2]
                j = head[c1]
                while j != EMPTY:
                    if j > i:
                        pos_j = positions[j] + rshift
                        r_ij = pos_i - pos_j
                        dist = np.linalg.norm(r_ij)
                        if dist < rcutoff and dist > 1e-12:
                            f_mag = compute_lj_force(dist, sigma, epsilon, rcutoff)
                            fij = f_mag * (r_ij / dist)
                            forces[i] += fij
                            forces[j] -= fij
                            potential_energy += compute_lj_potential(dist, sigma, epsilon, rcutoff)
                    j = lscl[j]
            i = lscl[i]
    return forces, potential_energy


def apply_berendsen_thermostat(velocities, target_temp, current_temp, dt, tau):
    lambda_factor = np.sqrt(1 + dt / tau * (target_temp / current_temp - 1))
    return velocities * lambda_factor

def apply_langevin_thermostat(velocities, gamma, dt, target_temp):
    kb = 1.380649e-23  #
    noise = np.random.normal(0, 1, size=velocities.shape)
    c1 = np.exp(-gamma * dt)
    c2 = np.sqrt((1 - c1**2) * kb * target_temp / mass_argon)
    return c1 * velocities + c2 * noise


def create_lattice(n_particles, box_size, dimensions):
    n_side = int(np.ceil(n_particles ** (1 / dimensions)))
    spacing = box_size / n_side
    positions = []
    for indices in np.ndindex(*([n_side] * dimensions)):
        if len(positions) < n_particles:
            pos = [(i + 0.5) * spacing for i in indices]
            noise = np.random.uniform(-0.05, 0.05, size=dimensions) * spacing
            positions.append(np.array(pos) + noise)
    return np.array(positions)

def initialize_velocities(n_particles, dimensions, target_temp):
    kb = 1.380649e-23  
    velocities = np.random.normal(0, 1, size=(n_particles, dimensions))
    velocities -= np.mean(velocities, axis=0)
    ke = 0.5 * mass_argon * np.sum(velocities**2)
    dof = n_particles * dimensions
    scale = np.sqrt(target_temp * kb * dof / (2 * ke))
    velocities *= scale
    return velocities


def compute_rdf(positions, box_size, rcutoff, n_bins):
    n_particles = positions.shape[0]
    dr = rcutoff / n_bins
    rdf_hist = np.zeros(n_bins)
    
    for i in range(n_particles):
        for j in range(i + 1, n_particles):
            rij = positions[i] - positions[j]
            rij -= box_size * np.round(rij / box_size)  # PBC
            r = np.linalg.norm(rij)
            if r < rcutoff:
                bin_index = int(r / dr)
                rdf_hist[bin_index] += 2

    density = n_particles / box_size**3
    r_values = dr * (np.arange(n_bins) + 0.5)
    shell_volumes = 4/3 * np.pi * ((r_values + dr/2)**3 - (r_values - dr/2)**3)
    ideal_counts = density * shell_volumes * n_particles
    g_r = rdf_hist / ideal_counts
    return r_values, g_r


def run_simulation(params, compute_rdf_flag=False, rdf_sample_steps=1000, n_bins=100):
    box_size = (params['n_particles'] * mass_argon / params['density']) ** (1 / params['dimensions'])
    positions = create_lattice(params['n_particles'], box_size, params['dimensions'])
    velocities = initialize_velocities(params['n_particles'], params['dimensions'], params['temperature'])
    dof = params['n_particles'] * params['dimensions']

    temp_list = []
    rdf_accum = np.zeros(n_bins)
    rdf_count = 0

    for step in tqdm(range(params['steps']), desc="Running Simulation"):
        forces, _ = compute_forces_lca(positions, box_size, params['rcutoff'],
                                       params['sigma'], params['epsilon'], use_pbc=True)
        velocities += 0.5 * forces / mass_argon * params['dt']
        positions += velocities * params['dt']
        positions %= box_size
        forces, _ = compute_forces_lca(positions, box_size, params['rcutoff'],
                                       params['sigma'], params['epsilon'], use_pbc=True)
        velocities += 0.5 * forces / mass_argon * params['dt']

        kinetic = 0.5 * mass_argon * np.sum(velocities ** 2)
        temp = 2 * kinetic / (dof * 1.380649e-23) 
        temp_list.append(temp)

        if params['thermostat_type'] == 'berendsen':
            velocities = apply_berendsen_thermostat(velocities, params['temperature'], temp, params['dt'], params['tau_ber'])
        elif params['thermostat_type'] == 'langevin':
            velocities = apply_langevin_thermostat(velocities, params['gamma_langevin'], params['dt'], params['temperature'])

        if compute_rdf_flag and step >= params['steps'] - rdf_sample_steps:
            r_vals, gr = compute_rdf(positions, box_size, params['rcutoff'], n_bins)
            rdf_accum += gr
            rdf_count += 1

    rdf_avg = rdf_accum / rdf_count if rdf_count > 0 else None
    return temp_list, r_vals, rdf_avg

params = {
    'dimensions': 3,
    'n_particles': 1000,
    'density': 1374,
    'dt': 2e-15,
    'steps': 1000,
    'temperature': 94.4,
    'sigma': sigma,
    'epsilon': epsilon,
    'rcutoff': 2.5 * sigma,
    'tau_ber': 1e-13,
    'gamma_langevin': 1e13,
    'thermostat_type': 'langevin'
}

_, r_vals, g_r = run_simulation(params, compute_rdf_flag=True)

if g_r is not None:
    plt.figure(figsize=(8, 5))
    plt.plot(r_vals * 1e10, g_r, label=f"T = {params['temperature']} K")
    plt.xlabel('r (Å)')
    plt.ylabel('g(r)')
    plt.title(f"Radial Distribution Function for Argon at {params['temperature']} K")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


Running Simulation:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 